## This notebook prepares the training data used in training GPT-2

In [1]:
import pandas as pd
from datetime import datetime, timedelta
import time
import json
import re
# pd.set_option('display.max_colwidth', None)
# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

IMDb movie data is pulled into a dataframe

In [2]:
df = pd.read_csv("data/IMDb movies.csv")

C:\Users\natha\Miniconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df.head(3)

,imdb_title_id,title,original_title,year,date_published,genre,duration,country,language,director,...,actors,description,avg_vote,votes,budget,usa_gross_income,worlwide_gross_income,metascore,reviews_from_users,reviews_from_critics
0,tt0000009,Miss Jerry,Miss Jerry,1894,1894-10-09,Romance,45,USA,None,Alexander Black,...,"Blanche Bayliss, William Courtenay, Chauncey D...",The adventures of a female reporter in the 1890s.,5.9,154,NaN,NaN,NaN,NaN,1.0,2.0
1,tt0000574,The Story of the Kelly Gang,The Story of the Kelly Gang,1906,1906-12-26,"Biography, Crime, Drama",70,Australia,None,Charles Tait,...,"Elizabeth Tait, John Tait, Norman Campbell, Be...",True story of notorious Australian outlaw Ned ...,6.1,589,$ 2250,NaN,NaN,NaN,7.0,7.0
2,tt0001892,Den sorte drøm,Den sorte drøm,1911,1911-08-19,Drama,53,"Germany, Denmark",NaN,Urban Gad,...,"Asta Nielsen, Valdemar Psilander, Gunnar Helse...",Two men of high rank are both wooing the beaut...,5.8,188,NaN,NaN,NaN,NaN,5.0,2.0


In [4]:
df.columns

Index(['imdb_title_id', 'title', 'original_title', 'year', 'date_published',
       'genre', 'duration', 'country', 'language', 'director', 'writer',
       'production_company', 'actors', 'description', 'avg_vote', 'votes',
       'budget', 'usa_gross_income', 'worlwide_gross_income', 'metascore',
       'reviews_from_users', 'reviews_from_critics'],
      dtype='object')

Filter out incomplete descriptions

In [5]:
temp = df.loc[df.description.notnull()].copy()
df = temp[~temp.description.str.contains("\.{3}$")].copy()

In [7]:
training_data = list(df["description"].values)

In [8]:
for i in training_data[:10]:
    print(i)

The adventures of a female reporter in the 1890s.
True story of notorious Australian outlaw Ned Kelly (1855-80).
The fabled queen of Egypt's affair with Roman general Marc Antony is ultimately disastrous for both of them.
Loosely adapted from Dante's Divine Comedy and inspired by the illustrations of Gustav Doré the original silent film has been restored and has a new score by Tangerine Dream.
The story of Madame DuBarry, the mistress of Louis XV of France, and her loves in the time of the French revolution.
An epic Italian film "Quo Vadis" influenced many of the later movies.
The movie depicts the Romanian War of Independence (1877-1878).
Richard of Gloucester uses manipulation and murder to gain the English throne.
After Dr. Friedrich's wife becomes mentally unstable and his research papers are rejected, he leaves the country to respite.
Inspector Juve is tasked to investigate and capture an infamous criminal Fantomas.


In [9]:
delim = "<|endoftext|>"

In [10]:
plots = open("plot_training.txt", "w", encoding='utf-8')

for i in training_data:
    plots.write("{}\n\n{}\n\n".format(i, delim))

plots.close()